# Import libraries

In [ ]:
try:
    import google.colab
    # Running on Google Colab, so install Biopython first
    !pip install biopython

except ImportError:
    pass

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 10.3 MB/s eta 0:00:00


In [ ]:
import argparse
import math
import requests #Used for HTTP request
import os
import time
import pandas as pd
import numpy as np
import urllib.request
from sklearn import preprocessing
from collections import Counter
from Bio import Entrez, SeqIO, ExPASy, SwissProt, GenBank
from Bio.SeqUtils.ProtParam import ProteinAnalysis



# Fetch Data


In [ ]:
# Step 1: Fetch Phage Data from PhagesDB
def fetch_phages_data():
    url = 'https://phagesdb.org/api/phages/'
    params = {
        'page': 1,
        'page_size': 5  # Adjust page_size as needed to fetch more data if necessary
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        # Filter out phages where sequencing is not finished
        complete_phages = [phage for phage in data['results'] if phage.get('seq_finished') is True]
        return {'results': complete_phages, 'count': data['count'], 'next': data['next'], 'previous': data['previous']}
    else:
        print(f"Failed to fetch data: {response.status_code}")
        return None


# Step 2: Parse GenBank Accession Numbers
def parse_genbank_accession(phages_data):
    accession_numbers = []
    for phage in phages_data['results']:
        if 'genbank_accession' in phage and phage['genbank_accession']:
            accession_numbers.append(phage['genbank_accession'])
    return accession_numbers

# Step 3: Fetch Detailed Information from GenBank
def fetch_genbank_data(accession_numbers):
    all_proteins = []
    all_protein_ids = []
    for accession in accession_numbers:
        url = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi'
        params = {
            'db': 'nucleotide',
            'id': accession,
            'rettype': 'gb',
            'retmode': 'text'
        }
        response = requests.get(url, params=params)
        if response.status_code == 200:

            genbank_data = response.text
            non_hypothetical_proteins = extract_non_hypothetical_proteins(genbank_data)
            all_proteins.extend(non_hypothetical_proteins)
        else:
            print(f"Failed to fetch data for {accession}: {response.status_code}")

    print(accession_numbers)
    for protein in all_proteins:
        p_id = protein.get('protein_id')
        p_product = protein.get('product')
        print(p_id, p_product)
        all_protein_ids.append(p_id)

    return all_protein_ids


def extract_non_hypothetical_proteins(genbank_data):
    lines = genbank_data.split("\n")
    proteins = []
    capturing = False
    current_protein = {}

    for line in lines:
        line = line.strip()
        if line.startswith("CDS"):
            if current_protein:  # save the previous one if not hypothetical
                if current_protein.get('product') and "hypothetical protein" not in current_protein['product']:
                    proteins.append(current_protein)
            current_protein = {'location': line.split()[1]}  # reset for the new CDS
            capturing = True
        elif capturing:
            if line.startswith("/"):
                key, value = line.split("=")[0], "=".join(line.split("=")[1:])
                key = key.replace("/", "").strip()
                value = value.replace('"', '').strip()
                current_protein[key] = value
            elif line.startswith("ORIGIN") or line.startswith("gene") and current_protein:  # end of CDS entry
                if 'product' in current_protein and "hypothetical protein" not in current_protein['product']:
                    proteins.append(current_protein)
                capturing = False
                current_protein = {}

    # Check for the last CDS entry
    if current_protein and 'product' in current_protein and "hypothetical protein" not in current_protein['product']:
        proteins.append(current_protein)

    return proteins

'''
# Main function to orchestrate the fetching and processing
def main():
    phages_data = fetch_phages_data()
    if phages_data:
        accession_numbers = parse_genbank_accession(phages_data)
        if accession_numbers:
            fetch_genbank_data(accession_numbers)
        else:
            print("No accession numbers found.")
    else:
        print("No phage data to process.")



if __name__ == "__main__":
    main()
'''

'\n# Main function to orchestrate the fetching and processing\ndef main():\n    phages_data = fetch_phages_data()\n    if phages_data:\n        accession_numbers = parse_genbank_accession(phages_data)\n        if accession_numbers:\n            fetch_genbank_data(accession_numbers)\n        else:\n            print("No accession numbers found.")\n    else:\n        print("No phage data to process.")\n\n\n\nif __name__ == "__main__":\n    main()\n'

# Chemical Analysis stats

In [ ]:
Chemi_stats = {
                   'A':{'C-': 3, 'H-': 7, 'O-': 2, 'N-': 1, 'S-': 0},
                   'C':{'C-': 3, 'H-': 7, 'O-': 2, 'N-': 1, 'S-': 1},
                   'D':{'C-': 4, 'H-': 7, 'O-': 4, 'N-': 1, 'S-': 0},
                   'E':{'C-': 5, 'H-': 9, 'O-': 4, 'N-': 1, 'S-': 0},
                   'F':{'C-': 9, 'H-': 11,'O-': 2, 'N-': 1, 'S-': 0},
                   'G':{'C-': 2, 'H-': 5, 'O-': 2, 'N-': 1, 'S-': 0},
                   'H':{'C-': 6, 'H-': 9, 'O-': 2, 'N-': 3, 'S-': 0},
                   'I':{'C-': 6, 'H-': 13,'O-': 2, 'N-': 1, 'S-': 0},
                   'K':{'C-': 6, 'H-': 14,'O-': 2, 'N-': 2, 'S-': 0},
                   'L':{'C-': 6, 'H-': 13,'O-': 2, 'N-': 1, 'S-': 0},
                   'M':{'C-': 5, 'H-': 11,'O-': 2, 'N-': 1, 'S-': 1},
                   'N':{'C-': 4, 'H-': 8, 'O-': 3, 'N-': 2, 'S-': 0},
                   'P':{'C-': 5, 'H-': 9, 'O-': 2, 'N-': 1, 'S-': 0},
                   'Q':{'C-': 5, 'H-': 10,'O-': 3, 'N-': 2, 'S-': 0},
                   'R':{'C-': 6, 'H-': 14,'O-': 2, 'N-': 4, 'S-': 0},
                   'S':{'C-': 3, 'H-': 7, 'O-': 3, 'N-': 1, 'S-': 0},
                   'T':{'C-': 4, 'H-': 9, 'O-': 3, 'N-': 1, 'S-': 0},
                   'V':{'C-': 5, 'H-': 11,'O-': 2, 'N-': 1, 'S-': 0},
                   'W':{'C-': 11,'H-': 12,'O-': 2, 'N-': 2, 'S-': 0},
                   'Y':{'C-': 9, 'H-': 11,'O-': 3, 'N-': 1, 'S-': 0}
                }

def physical_chemical_feature(sequence):
  # Implementation from predPHI
    seq_new=sequence.replace('X','').replace('U','').replace('B','').replace('Z','')
    CE = 'CHONS'

    count = Counter(seq_new)
    code = []

    for c in CE:
        abundance_c = 0
        for key in count:
            num_c = Chemi_stats[key][c]
            abundance_c += num_c * count[key]

        code.append(abundance_c)
    return(code)

def molecular_weight(seq):
  # Implementation from predPHI
    #seq_new=seq.replace('X','').replace('U','').replace('B','').replace('Z','')
    analysed_seq = ProteinAnalysis(seq)
    analysed_seq.monoisotopic = True
    mw = analysed_seq.molecular_weight()
    return mw

# Deprecrated stuff

Ignore this, I've found more efficient implementations

In [ ]:
def fetch_genomic_data(accession_ids):
    Entrez.email = email
    handle = Entrez.efetch(db="nuccore", id=accession_ids, rettype="gb", retmode="text")
    records = SeqIO.parse(handle, "genbank")
    return records

def fetch_protein_data(accession_ids):
    Entrez.email = email
    handle = Entrez.efetch(db="protein", id=accession_ids, rettype="gb", retmode="text")
    records = SeqIO.parse(handle, "gb")
    return records

def fetch_protein_sequence(accession):
    Entrez.email = email  # Set your email here
    handle = Entrez.efetch(db="protein", id=accession, rettype="fasta", retmode="text")
    record = handle.read()
    handle.close()
    # The record will contain the FASTA format which includes the sequence header and the sequence itself
    return record


# probably dont need this FUNCTION
def record_to_seq_list(records):
  # Go over records and return a list of only the protein AA sequences
  sequence_list = []
  for rec in records:
    sequence_list.append(rec.seq)

  return sequence_list


# Dataset Creation

In [ ]:
email = 'rnguye20@ucsc.edu'

def get_protein_translations(genome_accession, min_length=30, max_length=float('inf'), exclude_hypothetical=True):
    # Given a genome accession code, return a list of all protein translations
    Entrez.email = email

    handle = Entrez.efetch(db="nuccore", id=genome_accession, rettype="gb", retmode="text")
    record = SeqIO.read(handle, "genbank")
    protein_accession_codes = []
    protein_translation_sequences = []
    for feature in record.features:
        if feature.type == "CDS" and "protein_id" in feature.qualifiers:
            protein_id = feature.qualifiers["protein_id"][0]
            protein_translation = feature.qualifiers.get("translation", [""])[0]
            protein_length = len(protein_translation)
            protein_description = feature.qualifiers.get("product", [""])[0]
            if exclude_hypothetical and "hypothetical" in protein_description.lower(): # exclude hypotheticals
                continue
            if protein_description.lower() == 'n/a': # exclude proteins that aren't named
                continue
            if min_length <= protein_length <= max_length: # exclude proteins that are under 30 sequences long
                protein_accession_codes.append(protein_id)
                protein_translation_sequences.append(protein_translation.replace('X','').replace('U','').replace('B','').replace('Z',''))


    if protein_translation_sequences:
      return protein_translation_sequences

In [ ]:
import time
## Feature extraction

def analyze_seq_list(sequence_list):
  # Go over each sequence and return a list of analyzed sequence objects
  analyzed_sequences = [ProteinAnalysis(seq) for seq in sequence_list]

  return analyzed_sequences

def count_CHONS(AAC_dict):
  # Given a dictionary of amino acid counts for a sequence, return the elemental composition
  # as a pandas dataframe

   # Elements denoted by '-' to avoid confusion with amino acid codes
  comp_dict = Counter({'C-':0, 'H-':0, 'O-':0, 'N-':0, 'S-':0})

  for key, value in AAC_dict.items():
    # For a count of amino acid, multiply the count by its known chemical composition
    # Then add it to the total
    chem_comp_dict = Chemi_stats[key]
    multiplied_dict = Counter({AA:element*value for (AA, element) in chem_comp_dict.items()})
    comp_dict += multiplied_dict

  return pd.DataFrame(comp_dict, index=[0])

def dataframe_average(dataframe_list):
  # Given a list of dataframes with equal dimensions, return a dataframe
  # consisting of element-wise calculated averaged values

  averaged_df = sum(dataframe_list)/len(dataframe_list)

  labels = list(averaged_df.columns)
  avg_label = [x+'_avg' for x in labels] # Label for avg
  averaged_df.columns = avg_label

  return averaged_df

def dataframe_std(dataframe_list):
  # Given a list of dataframes with equal dimensions, return a dataframe
  # consisting of element-wise calculated standard deviation values

  std_combined_df = pd.concat(dataframe_list, axis=0)  # Concatenate along rows
  std_inbetween_df = std_combined_df.std(axis=0)
  elementwise_std_df = pd.DataFrame(std_inbetween_df)
  std_df = pd.DataFrame(elementwise_std_df).T

  labels = list(std_df.columns)
  std_label = [x+'_std' for x in labels] #label for std
  std_df.columns = std_label

  return std_df

def dataframe_variance(dataframe_list):
  # Given a list of dataframes with equal dimensions, return a dataframe
  # consisting of element-wise calculated variance values

  var_combined_df = pd.concat(dataframe_list, axis=0)  # Concatenate along rows
  var_inbetween_df = var_combined_df.var(axis=0)
  elementwise_var_df = pd.DataFrame(var_inbetween_df)
  var_df = pd.DataFrame(elementwise_var_df).T

  labels = list(var_df.columns)
  var_label = [x+'_var' for x in labels] #label for var
  var_df.columns = var_label

  return var_df


#-------------------------------------------------------------------------------------------------

def feature_extraction(analyzed_list):
  # Return features for a phage or a host when given a list of ProteinAnalysis sequence objects
  # Returned dataframe is 1x79
  DF_seq = []
  molecular_weights = []

  for an_seq in analyzed_list:
    # Get list of each protein's AAP as a dataframe with
    # a column for molecular weight and 5 columns for CHONS
    AAC_dict = an_seq.count_amino_acids()
    AAP_dict = an_seq.get_amino_acids_percent()

    mw = an_seq.molecular_weight()

    chons_df = count_CHONS(AAC_dict)

    dataframed_dict = pd.DataFrame(AAP_dict, index=[0]) # Turn dictionary into dataframe (20 features)
    dataframed_dict.insert(len(dataframed_dict.columns), 'MolW', mw) # Add column for mw (1 feature)
    final_df = pd.concat([dataframed_dict, chons_df], axis=1) # Add CHONS counts (5 features)
    DF_seq.append(final_df) # dataframe is appended to list of dataframes for statistics calculations

  # Statistics stuff, applied over the list of dataframes
  avg_df = dataframe_average(DF_seq)
  std_df = dataframe_std(DF_seq)
  var_df = dataframe_variance(DF_seq)

  feature_dataframe = pd.concat([avg_df, std_df, var_df], axis=1) #concatenate df horizontally

  return feature_dataframe




def relabel_dataframe(df, phage=True):
  # Relabel dataframe columns to specify which features belong to phage or host
  suffix = '_p' if phage else '_h'
  label = list(df)
  new_label = [x+suffix for x in label]
  df.columns = new_label

#-------------------------------------------------------------------------------------------------

def accession_to_features(accession, phage=True):
  # For some reason this process takes waaay too long
  protein_sequence_list = get_protein_translations(accession)

  if protein_sequence_list: # If we even got anything from the accession code
    analyzed_list = analyze_seq_list(protein_sequence_list)

    final_dataframe = feature_extraction(analyzed_list)
    relabel_dataframe(final_dataframe, phage) # label dataframe based on whether phage or host

    entity_category = 'phage' if phage else 'host' # label for organism type
    final_dataframe.insert(0, entity_category, accession)

    return final_dataframe



# Testing

In [ ]:
# EXAMPLE
accession_phage = 'NC_023604'
accession_host = 'LC361338'
#print(accession_to_features(accession_phage))
print(accession_to_features(accession_host, phage=False))
#print(get_protein_translations(accession_host))

       host   A_avg_h  C_avg_h   D_avg_h   E_avg_h   F_avg_h   G_avg_h  \
0  LC361338  0.151515      0.0  0.047475  0.024242  0.025253  0.129293   

    H_avg_h   I_avg_h   K_avg_h  ...  T_var_h  V_var_h  W_var_h  Y_var_h  \
0  0.017172  0.024242  0.009091  ...      NaN      NaN      NaN      NaN   

   MolW_var_h  C-_var_h  H-_var_h  O-_var_h  N-_var_h  S-_var_h  
0         NaN       NaN       NaN       NaN       NaN       NaN  

[1 rows x 79 columns]


In [ ]:
# Grab test_set.csv and training_set.csv (list of phage-host interactions) into reads into datafile
# Use this to create ALL features for each phage and host
accession_url = 'https://raw.githubusercontent.com/xialab-ahu/PredPHI/master/data/test_set.csv'
accession2_url = 'https://raw.githubusercontent.com/xialab-ahu/PredPHI/master/data/training_set.csv'
test_set = 'test_set.csv'
train_set = 'training_set.csv'

fields = ["phage", "host", "class"];

urllib.request.urlretrieve(accession_url, test_set)
df = pd.read_csv("test_set.csv", sep="\t")

urllib.request.urlretrieve(accession2_url, train_set)
df2 = pd.read_csv("training_set.csv", sep="\t")

# Just some testing code, ignore
interact_list = []
phage_list = []
host_list = []

In [ ]:
total_df = pd.concat([df,df2], axis=0) # HAS ALL INTERACTIONS, USE THIS ONE
phage_list = total_df['phage'].tolist()
host_list = total_df['host'].tolist()

phage_list = set(phage_list)
host_list = set(host_list)

print(len(phage_list), 'phages')
print(len(host_list), 'hosts')
print(len(phage_list)+len(host_list), 'phages+hosts')
print(len(total_df), 'total interactions')

3449 phages
301 hosts
3750 phages+hosts
615340 total interactions


# Quick overview of how it should probably go
Ok, so since we have a list of all accession codes for phages and hosts, run accession_to_features(accession, phage=True/False) over all of them, then store the accession code and resulting feature dataframe as a key:value in a dict.

After we have the features stored, iterate through the interaction dataframe (total_df) and using the accession codes as keys, grab the feature dataframes of both from the dict then concatenate the dataframes together. Add a column for the classifcation (whether or not they interact).


>phage_feat = feature_dict[phage_accession]

>host_feat = feature_dict[host_accession]

>final_feature_dataframe = df.concat([phage_feat, host_feat], axis=1)

>dataframed_dict.insert(0, 'interaction', classification)

classification being either a 1 (true) or a 0 (false)

With the final, concatenated dataframe, write it into a line in a .csv file. After we run through the entire interactions list, this will be our final data set.

In [ ]:
# This process takes a gazillion years
feature_dict = {}
accession_errors = []

for accession in phage_list.union(host_list):
    is_phage = accession in phage_list
    try:
      feature_dict[accession] = accession_to_features(accession, phage=is_phage)
    except Exception as e:
      print("Had some trouble with this accession code:", accession)
      accession_errors.append(accession)

In [ ]:
# This process is a little faster
final_df_list = []

for index, row in total_df.iterrows():
  if row['phage'] in feature_dict.keys() and row['host'] in feature_dict.keys():
    phage_feat = feature_dict[row['phage']]
    host_feat = feature_dict[row['host']]

    if isinstance(phage_feat, pd.DataFrame) and isinstance(host_feat, pd.DataFrame):
      combined_features_df = pd.concat([phage_feat.reset_index(drop=True),
                                        host_feat.reset_index(drop=True)], axis=1)
      combined_features_df['interaction'] = row['class']
      final_df_list.append(combined_features_df)

# Concatenate all interaction feature dataframes into a single dataframe
final_features_df = pd.concat(final_df_list, axis=0, ignore_index=True)

final_features_df.to_csv('final_interaction_features.csv', index=False)

In [ ]:
print(len(final_df_list))

417529
